# Analysing Discounts Part3

## 1. Imports of libraries and data

In [ ]:
import pandas as pd

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
o_ol_pro_m_original = pd.read_csv("o_ol_pro_m.csv")

FileNotFoundError: ignored

In [ ]:
o_ol_pro_m_original["created_date"] = pd.to_datetime(o_ol_pro_m_original["created_date"])

In [ ]:
o_ol_pro_m_original["date"] = pd.to_datetime(o_ol_pro_m_original["date"])

In [ ]:
#o_ol_pro_m_original.info()

In [ ]:
o_ol_pro_m = o_ol_pro_m_original.copy()

In [ ]:
o_ol_pro_m["price_total"] = o_ol_pro_m.price * o_ol_pro_m.product_quantity

## 2. Looking at time

The board of directors have the opion that the revenue has been decreasing in the last quarter and that the number of products sold did go up. Can we verify this statement with the data we have? 

In [ ]:
#what is the latest order? -- 2018-03-14 13:58:36
o_ol_pro_m.loc[o_ol_pro_m.created_date == o_ol_pro_m["created_date"].max(), ['order_id', 'created_date']]

So the latest order was in Q1 2018 - for this quarter we do not have all information yet. 
The quarter to which the executives are thus referring is Q4 2017. 
I'll extract the data for Q4-2017 and verify:
* what is the revenue?
* what is total number of products sold? 

In [ ]:
o_ol_pro_m_last_quarter = o_ol_pro_m.loc[(o_ol_pro_m["created_date"].dt.strftime("%Y") == "2017") & (o_ol_pro_m["created_date"].dt.quarter == 4) , :].copy()

o_ol_pro_m_last_quarter.unit_price_total.sum().round(2), o_ol_pro_m_last_quarter.product_quantity.sum(), 

In Q4 2017 we had sales of 3,037,096.34 and a total number of products sold of 20,251.

* 2017 - Q4: (3,037,096.34, 20,251)
* 2017 - Q3: (1,366,908.1,  10,429)
* 2017 - Q2: (  845,881.28,   6,725)
* 2017 - Q1: (  961,560.19,  10,013)
* 2016 - Q4: (0.0, 0)
* 2016 - Q3: (0.0, 0)

And looking forward:

* 2018 - Q1: (1,605,758.49, 12,593) 

Q1 is not yet finished and will be less than then 3 million of Q4 2017.
But it passed already 1 Million and is thus higher than all other quarters before.


In [ ]:
bins_quarter = []

In [ ]:
#df_plot= pd.pivot_table(df_test, columns='VehicleType', index=['hour'], aggfunc='sum', values='Minutes_Idling').plot(kind='bar')

## 3. Looking at different discounts (in € and in %)

In [ ]:
bins = [-300, 0, 25, 50, 75, 100, 150, 300, 2000]

In [ ]:
o_ol_pro_m["binned_discount_in_EUR"] = pd.cut(o_ol_pro_m.discount, bins)
o_ol_pro_m.info()

In [ ]:
sns.catplot(data=o_ol_pro_m, 
            kind="bar",
            x="binned_discount_in_EUR", 
            y="unit_price_total",
            aspect = 3,
            estimator = sum
            );

## 4. Looking at broad categories'

In [ ]:
sns.catplot(data=o_ol_pro_m, 
            x="category", 
            y="unit_price_total",
            aspect = 5
            );

In [ ]:
sns.catplot(data=o_ol_pro_m, 
            kind="bar",
            x="broad_category", 
            y="unit_price_total",
            aspect = 3,
            estimator = sum
            );

### Aggressive Discounts

In [ ]:
aggr_bins = [0, 20, 50, 2000]
o_ol_pro_m["aggressive_discount_perc"] = pd.cut(o_ol_pro_m["%_discount"], aggr_bins)
o_ol_pro_m.info()

In [ ]:
sns.catplot(data=o_ol_pro_m, 
            x="broad_category", 
            y="unit_price_total",
            hue="aggressive_discount_perc",
            aspect = 2.5,
            );

In [ ]:
o_ol_pro_m.head()

In [ ]:
discounted_accessories_with_high_percentage = o_ol_pro_m.loc[(o_ol_pro_m["%_discount"] >= 50) & (o_ol_pro_m.broad_category == "Accessories"), :]

In [ ]:
sns.catplot(data=discounted_accessories_with_high_percentage, 
            x="category", 
            y="unit_price_total",
            hue="binned_discount_in_EUR",
            aspect = 2.5,
            );



There are agressive discounts above 50%, which also have a high value discount of above 75 EUR / or 100 Eur. 
They are mainly in selling watches, speakers and keyboards.

### crazy analysis - discount percentage and total discount per broad category --- No big findings

In [ ]:
sns.catplot(data=o_ol_pro_m, 
            x="binned_discount_in_EUR", 
            y="%_discount",
            aspect = 5,
            hue = "broad_category"
            );

## 5. Analysis of categories with an overview of total_revenue and total_discount

In [ ]:
#category vs revenue

#total_revenue = o_ol_pro_m.resample("M", on="created_date").unit_price_total.mean()
#total_revenue.plot(kind='bar');


In [ ]:
#category vs revenue
df_revenue = o_ol_pro_m.groupby("category").unit_price_total.sum().sort_values(ascending=False)
df_revenue.plot(kind='bar');

In [ ]:
df_discount = o_ol_pro_m.groupby("category").total_discount.sum().sort_values(ascending=False)
df_discount.plot(kind='bar');

In [ ]:
df_discount_percent = o_ol_pro_m.groupby("category")["%_discount"].mean().sort_values(ascending=False)
df_discount_percent.plot(kind='bar');

### Classification

In [ ]:
# calculating %_discount for each category
# why are there differences?
# what categories have more discount in percent? and also in absolute terms ("discount_total")?

categories_discount = o_ol_pro_m.groupby(["broad_category", "category"]).agg({"price": "mean", "price_total": "sum", "unit_price" : "mean", "total_discount" : ["mean", "sum"], "unit_price_total" : ["mean", "sum"], "product_quantity" : "sum", "%_discount" : ["mean", "sum"], "discount" : ["sum", "mean"]}).copy()
 
    
    

In [ ]:
#divided by price
categories_discount["%_discount_total_price"] = categories_discount[("total_discount", "sum")] / categories_discount[("price_total", "sum")]  * 100
#the quantity will be missing


In [ ]:
#show
#categories_discount


#### Three Classifications of Broad_Categories
**1. More revenue (absolut and mean) and more discount --> this is also a nice category, at least we have more revenue**
* Accessories

**2. More revenue and less than average discount (21%) --> this is a category we will be in**
* Harddrive/Flashdrive
* iPhones/Tablets

**3. Less revenue and more discount --> this one is not so desirable**
* Service_Used_Other


**4. Less revenue (compared to other broad categories) and less discount** --> maybe irrelevant
* Monitors_Laptops_Computers






In [ ]:
#sort by revenue_absolute
categories_discount.nlargest(21, ("unit_price_total", "sum"))

In [9]:
#categories_discount["%_discount_total_price"].mean() # mean percentage of the categories

#### Three Classifications of Categories:
**1. More revenue (absolut and mean) and more discount --> this is also a nice category, at least we have more revenue**
* Keyboard
* Accessories
* Second-Hand


**2. More revenue and less than average discount (21%) --> this is a category we will be in**
* Harddrive
* iPhone
* Monitor
* Tablet
* Computer
* RAM
* Watches
* Other


**3. Less revenue and more discount --> this one is not so desirable**
* Headphones
* Flashdrive
* Powerbank



For Harddrive/Flashdrive we want to see how much money we are making: 2.2 Million (unit_price_total.sum) 
and then compare the absolute discounts: 430.000  --> 16% 

    2,207,561.01 € Revenue (unit_price_total.sum) 
       437120.53 € Discounts (total_discount.sum)
           16.5 % Discount


In [ ]:
#sort by revenue_mean

Sai:
    price 500 EUR
    20% discount
    new price sold at 400 EUR (unit price)
    
    discount from the lower price is higher.
    100/400 --> 25%
    
    100/500 --> 20%
    
    yaaaaaaaaaaayyyyyyyyyyyy
    
    PARTY YAAAAAAAAYYYYYY
    
    